In [201]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

from catboost import CatBoostClassifier

from skopt import BayesSearchCV

from xgboost import XGBClassifier

import pickle

import animalhelper as ah

%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:
!ls data

aac_intakes.csv          df_dog.csv               master12218.csv
aac_intakes_outcomes.csv df_mod.csv               master_df.csv
aac_outcomes.csv         engineered.csv           master_df_1128.csv
all_categoricals.csv     group_shade.csv          master_df_1129.csv
concat_backup.csv        hour_ids.csv             model12218.csv
df_cat.csv               hr_name.csv


In [139]:
df = pd.read_csv('./data/aac_intakes_outcomes.csv')

In [233]:
df2 = pd.read_csv('./data/aac_intakes.csv')

In [235]:
df2 = df2[['animal_id', 'name']]

In [236]:
name_freqs = pickle.load(open('name_freqs.pkl', 'rb'))

In [238]:
def get_name_frequencies(x):
    return name_freqs[x]

In [240]:
df2['name_freqs'] = df2['name'].apply(lambda x: get_name_frequencies(x))

In [242]:
df2 = df2.drop(labels=['name'], axis=1)

In [243]:
df2.head()

,animal_id,name_freqs
0,A706918,52
1,A724273,1
2,A665644,0
3,A682524,22
4,A743852,32


In [140]:
df.head()

,age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,...,age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days
0,10 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,...,"(7.5, 10.0]",2017-12-07 00:00:00,12,2017,2017-12,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194
1,7 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,...,"(5.0, 7.5]",2014-12-19 10:21:00,12,2014,2014-12,Friday,10,2.0,1 days 06:14:00.000000000,1.259722
2,6 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2190,6.0,"(5.0, 7.5]",2014-03-08 17:10:00,...,"(5.0, 7.5]",2014-03-07 14:26:00,3,2014,2014-03,Friday,14,3.0,1 days 02:44:00.000000000,1.113889
3,10 years,A047759,2004-04-02 00:00:00,Partner,Transfer,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,...,"(7.5, 10.0]",2014-04-02 15:55:00,4,2014,2014-04,Wednesday,15,1.0,4 days 23:17:00.000000000,4.970139
4,16 years,A134067,1997-10-16 00:00:00,NaN,Return to Owner,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,...,"(15.0, 17.5]",2013-11-16 09:02:00,11,2013,2013-11,Saturday,9,1.0,0 days 02:52:00.000000000,0.119444


In [141]:
df.columns.values

array(['age_upon_outcome', 'animal_id_outcome', 'date_of_birth',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome',
       'age_upon_outcome_(days)', 'age_upon_outcome_(years)',
       'age_upon_outcome_age_group', 'outcome_datetime', 'outcome_month',
       'outcome_year', 'outcome_monthyear', 'outcome_weekday',
       'outcome_hour', 'outcome_number', 'dob_year', 'dob_month',
       'dob_monthyear', 'age_upon_intake', 'animal_id_intake',
       'animal_type', 'breed', 'color', 'found_location',
       'intake_condition', 'intake_type', 'sex_upon_intake', 'count',
       'age_upon_intake_(days)', 'age_upon_intake_(years)',
       'age_upon_intake_age_group', 'intake_datetime', 'intake_month',
       'intake_year', 'intake_monthyear', 'intake_weekday', 'intake_hour',
       'intake_number', 'time_in_shelter', 'time_in_shelter_days'],
      dtype=object)

In [142]:
df = df.iloc[:, 18:]

In [143]:
df2 = pd.read_csv('./data/aac_intakes_outcomes.csv')

In [144]:
df = pd.concat([df, df2['outcome_type']], axis=1)

In [145]:
len(df['animal_id_intake'].unique())

71961

In [146]:
df = df.drop_duplicates(subset='animal_id_intake')

In [147]:
df = df.drop(labels=['found_location', 'count', 'age_upon_intake_(years)', 'intake_month', 'intake_year', 
                'intake_monthyear', 'intake_number', 'time_in_shelter'], axis=1)

In [148]:
df = df[(df['animal_type'] == 'Dog') | (df['animal_type'] == 'Cat')]

In [149]:
df['intake_datetime'] = pd.to_datetime(df['intake_datetime'])

In [150]:
df.columns.values

array(['dob_monthyear', 'age_upon_intake', 'animal_id_intake',
       'animal_type', 'breed', 'color', 'intake_condition', 'intake_type',
       'sex_upon_intake', 'age_upon_intake_(days)',
       'age_upon_intake_age_group', 'intake_datetime', 'intake_weekday',
       'intake_hour', 'time_in_shelter_days', 'outcome_type'],
      dtype=object)

In [151]:
df['intake_hour'] = df['intake_datetime'].dt.hour

In [167]:
df = df.drop(labels=['age_upon_intake'], axis=1)

KeyError: "labels ['age_upon_intake'] not contained in axis"

In [168]:
df = df.drop(labels=['dob_monthyear'], axis=1)

## Reduce color to basic colors

In [153]:
df['color'] = df['color'].apply(lambda x: ah.color(x))

## Group breeds

In [154]:
df['breed'] = df['breed'].apply(lambda x: ah.group_dogs(x, False))

In [155]:
df_cat = df[df['animal_type'] == 'Cat']

In [156]:
df_cat['breed'] = df_cat['breed'].apply(lambda x: ah.cat_breed(x, False))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [157]:
df = pd.concat([df, df_cat])

In [158]:
df.columns.values

array(['dob_monthyear', 'animal_id_intake', 'animal_type', 'breed',
       'color', 'intake_condition', 'intake_type', 'sex_upon_intake',
       'age_upon_intake_(days)', 'age_upon_intake_age_group',
       'intake_datetime', 'intake_weekday', 'intake_hour',
       'time_in_shelter_days', 'outcome_type'], dtype=object)

In [159]:
df = df.drop_duplicates(subset='animal_id_intake', keep='last')

In [160]:
df['breed'].value_counts()

short hair       27060
Terrier           9524
Toy               7376
Sporting          6529
Herding           5897
Working           4604
Hound             2996
Non-Sporting      1871
long hair         1243
Terrier & Toy      111
Unknown              1
Name: breed, dtype: int64

In [161]:
def fix_breed(s):
    if s == 'Terrier & Toy':
        return 'Terrier'
    else:
        return s

In [162]:
df['breed'] = df['breed'].apply(lambda x: fix_breed(x))

In [163]:
df['breed'].value_counts()

short hair      27060
Terrier          9635
Toy              7376
Sporting         6529
Herding          5897
Working          4604
Hound            2996
Non-Sporting     1871
long hair        1243
Unknown             1
Name: breed, dtype: int64

In [164]:
df = df[df['breed'] != 'Unknown']

In [169]:
df.head()

,animal_id_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,age_upon_intake_(days),age_upon_intake_age_group,intake_datetime,intake_weekday,intake_hour,time_in_shelter_days,outcome_type
0,A006100,Dog,Sporting,Yellow,Normal,Stray,Neutered Male,3650,"(7.5, 10.0]",2017-12-07 00:00:00,Thursday,0,0.588194,Return to Owner
3,A047759,Dog,Hound,Tricolor,Normal,Owner Surrender,Neutered Male,3650,"(7.5, 10.0]",2014-04-02 15:55:00,Wednesday,15,4.970139,Transfer
4,A134067,Dog,Herding,Brown,Injured,Public Assist,Neutered Male,5840,"(15.0, 17.5]",2013-11-16 09:02:00,Saturday,9,0.119444,Return to Owner
5,A141142,Dog,Sporting,Black,Aged,Stray,Spayed Female,5475,"(12.5, 15.0]",2013-11-16 14:46:00,Saturday,14,0.870833,Return to Owner
6,A163459,Dog,Terrier,Black,Normal,Stray,Intact Female,5475,"(12.5, 15.0]",2014-11-14 15:11:00,Friday,15,0.178472,Return to Owner


## Add in name frequencies

In [246]:
df = df2.merge(df, left_on='animal_id', right_on='animal_id_intake')

In [250]:
df = df.drop_duplicates()

## Remove rare occurences of features and response

In [176]:
df = df[(df['outcome_type'] != 'Rto-Adopt') & (df['outcome_type'] != 'Missing') & (df['outcome_type'] != 'Disposal')]

In [180]:
df = df[df['intake_type'] != 'Euthanasia Request']

In [181]:
df.head()

,animal_id_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,age_upon_intake_(days),age_upon_intake_age_group,intake_datetime,intake_weekday,intake_hour,time_in_shelter_days,outcome_type
0,A006100,Dog,Sporting,Yellow,Normal,Stray,Neutered Male,3650,"(7.5, 10.0]",2017-12-07 00:00:00,Thursday,0,0.588194,Return to Owner
3,A047759,Dog,Hound,Tricolor,Normal,Owner Surrender,Neutered Male,3650,"(7.5, 10.0]",2014-04-02 15:55:00,Wednesday,15,4.970139,Transfer
4,A134067,Dog,Herding,Brown,Injured,Public Assist,Neutered Male,5840,"(15.0, 17.5]",2013-11-16 09:02:00,Saturday,9,0.119444,Return to Owner
5,A141142,Dog,Sporting,Black,Aged,Stray,Spayed Female,5475,"(12.5, 15.0]",2013-11-16 14:46:00,Saturday,14,0.870833,Return to Owner
6,A163459,Dog,Terrier,Black,Normal,Stray,Intact Female,5475,"(12.5, 15.0]",2014-11-14 15:11:00,Friday,15,0.178472,Return to Owner


## Model

In [251]:
df_model = df.copy()

In [252]:
df_model = df_model.drop(labels=['animal_id_intake', 'intake_datetime', 'time_in_shelter_days'], axis=1)

In [254]:
df_model = df_model.drop(labels=['animal_id'], axis=1)

In [255]:
df_model.head()

,name_freqs,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,age_upon_intake_(days),age_upon_intake_age_group,intake_weekday,intake_hour,outcome_type
0,52,Dog,Sporting,White,Normal,Stray,Spayed Female,2920,"(7.5, 10.0]",Sunday,12,Return to Owner
1,1,Dog,Hound,Sable,Normal,Stray,Intact Male,330,"(-0.025, 2.5]",Thursday,18,Return to Owner
2,0,Cat,short hair,Tricolor,Sick,Stray,Intact Female,28,"(-0.025, 2.5]",Monday,7,Transfer
3,22,Dog,Working,Yellow,Normal,Stray,Neutered Male,1460,"(2.5, 5.0]",Sunday,10,Return to Owner
4,32,Dog,Sporting,Brown,Normal,Owner Surrender,Neutered Male,730,"(-0.025, 2.5]",Saturday,12,Return to Owner


In [256]:
def simplify_response(s):
    if s == 'Adoption':
        return s
    else:
        return 'Not Adoption'

In [257]:
df_model['outcome_type'] = df_model['outcome_type'].apply(lambda x: simplify_response(x))

In [258]:
df_model = df_model.dropna()

In [259]:
df_model.head()

,name_freqs,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,age_upon_intake_(days),age_upon_intake_age_group,intake_weekday,intake_hour,outcome_type
0,52,Dog,Sporting,White,Normal,Stray,Spayed Female,2920,"(7.5, 10.0]",Sunday,12,Not Adoption
1,1,Dog,Hound,Sable,Normal,Stray,Intact Male,330,"(-0.025, 2.5]",Thursday,18,Not Adoption
2,0,Cat,short hair,Tricolor,Sick,Stray,Intact Female,28,"(-0.025, 2.5]",Monday,7,Not Adoption
3,22,Dog,Working,Yellow,Normal,Stray,Neutered Male,1460,"(2.5, 5.0]",Sunday,10,Not Adoption
4,32,Dog,Sporting,Brown,Normal,Owner Surrender,Neutered Male,730,"(-0.025, 2.5]",Saturday,12,Not Adoption


In [260]:
X = df_model.iloc[:, :10]

In [261]:
y = df_model.iloc[:, -1]

In [262]:
le = LabelEncoder()
y = le.fit_transform(y)

In [263]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2325)

In [264]:
X_train.columns.values

array(['name_freqs', 'animal_type', 'breed', 'color', 'intake_condition',
       'intake_type', 'sex_upon_intake', 'age_upon_intake_(days)',
       'age_upon_intake_age_group', 'intake_weekday'], dtype=object)

In [267]:
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_train, y_train)

In [269]:
c = CatBoostClassifier(cat_features=[1, 2, 3, 4, 5, 6, 8, 9], depth=8, learning_rate = 0.23,
                   n_estimators = 800, one_hot_max_size=24, od_type = 'IncToDec', od_pval=1e-5,
                       l2_leaf_reg=1, eval_metric='AUC', class_weights=[1.1887, 1])

c.fit(X_train_val, y_train_val, use_best_model=True, eval_set=(X_test_val, y_test_val))

0:	test: 0.7274446	best: 0.7274446 (0)	total: 106ms	remaining: 1m 24s
1:	test: 0.7474751	best: 0.7474751 (1)	total: 147ms	remaining: 58.8s
2:	test: 0.7487522	best: 0.7487522 (2)	total: 189ms	remaining: 50.1s
3:	test: 0.7531361	best: 0.7531361 (3)	total: 209ms	remaining: 41.6s
4:	test: 0.7551965	best: 0.7551965 (4)	total: 252ms	remaining: 40.1s
5:	test: 0.7581246	best: 0.7581246 (5)	total: 294ms	remaining: 38.9s
6:	test: 0.7586324	best: 0.7586324 (6)	total: 316ms	remaining: 35.8s
7:	test: 0.7595475	best: 0.7595475 (7)	total: 358ms	remaining: 35.5s
8:	test: 0.7619672	best: 0.7619672 (8)	total: 408ms	remaining: 35.9s
9:	test: 0.7621240	best: 0.7621240 (9)	total: 431ms	remaining: 34s
10:	test: 0.7622548	best: 0.7622548 (10)	total: 451ms	remaining: 32.4s
11:	test: 0.7639670	best: 0.7639670 (11)	total: 485ms	remaining: 31.9s
12:	test: 0.7640336	best: 0.7640336 (12)	total: 518ms	remaining: 31.4s
13:	test: 0.7642805	best: 0.7642805 (13)	total: 577ms	remaining: 32.4s
14:	test: 0.7652703	best: 0

121:	test: 0.7723476	best: 0.7724026 (114)	total: 4.62s	remaining: 25.7s
122:	test: 0.7723476	best: 0.7724026 (114)	total: 4.65s	remaining: 25.6s
123:	test: 0.7723481	best: 0.7724026 (114)	total: 4.68s	remaining: 25.5s
124:	test: 0.7723492	best: 0.7724026 (114)	total: 4.71s	remaining: 25.4s
125:	test: 0.7723433	best: 0.7724026 (114)	total: 4.77s	remaining: 25.5s
126:	test: 0.7723433	best: 0.7724026 (114)	total: 4.79s	remaining: 25.4s
127:	test: 0.7723468	best: 0.7724026 (114)	total: 4.82s	remaining: 25.3s
128:	test: 0.7723484	best: 0.7724026 (114)	total: 4.88s	remaining: 25.4s
129:	test: 0.7723513	best: 0.7724026 (114)	total: 4.9s	remaining: 25.2s
130:	test: 0.7723567	best: 0.7724026 (114)	total: 4.92s	remaining: 25.1s
131:	test: 0.7723570	best: 0.7724026 (114)	total: 4.94s	remaining: 25s
132:	test: 0.7723598	best: 0.7724026 (114)	total: 4.96s	remaining: 24.9s
133:	test: 0.7723884	best: 0.7724026 (114)	total: 4.99s	remaining: 24.8s
134:	test: 0.7724253	best: 0.7724253 (134)	total: 5.02

239:	test: 0.7726024	best: 0.7726024 (239)	total: 8.88s	remaining: 20.7s
240:	test: 0.7725935	best: 0.7726024 (239)	total: 8.92s	remaining: 20.7s
241:	test: 0.7726011	best: 0.7726024 (239)	total: 8.96s	remaining: 20.7s
242:	test: 0.7726015	best: 0.7726024 (239)	total: 8.99s	remaining: 20.6s
243:	test: 0.7726149	best: 0.7726149 (243)	total: 9.02s	remaining: 20.6s
244:	test: 0.7726120	best: 0.7726149 (243)	total: 9.05s	remaining: 20.5s
245:	test: 0.7726077	best: 0.7726149 (243)	total: 9.09s	remaining: 20.5s
246:	test: 0.7726075	best: 0.7726149 (243)	total: 9.13s	remaining: 20.4s
247:	test: 0.7725994	best: 0.7726149 (243)	total: 9.16s	remaining: 20.4s
248:	test: 0.7725988	best: 0.7726149 (243)	total: 9.18s	remaining: 20.3s
249:	test: 0.7725866	best: 0.7726149 (243)	total: 9.22s	remaining: 20.3s
250:	test: 0.7725843	best: 0.7726149 (243)	total: 9.25s	remaining: 20.2s
251:	test: 0.7725841	best: 0.7726149 (243)	total: 9.28s	remaining: 20.2s
252:	test: 0.7725839	best: 0.7726149 (243)	total: 9

353:	test: 0.7725984	best: 0.7726149 (243)	total: 12s	remaining: 15.1s
354:	test: 0.7725340	best: 0.7726149 (243)	total: 12.1s	remaining: 15.1s
355:	test: 0.7725315	best: 0.7726149 (243)	total: 12.1s	remaining: 15.1s
356:	test: 0.7725308	best: 0.7726149 (243)	total: 12.1s	remaining: 15s
357:	test: 0.7725305	best: 0.7726149 (243)	total: 12.1s	remaining: 15s
358:	test: 0.7725289	best: 0.7726149 (243)	total: 12.2s	remaining: 14.9s
359:	test: 0.7725257	best: 0.7726149 (243)	total: 12.2s	remaining: 14.9s
360:	test: 0.7725284	best: 0.7726149 (243)	total: 12.2s	remaining: 14.8s
361:	test: 0.7725270	best: 0.7726149 (243)	total: 12.2s	remaining: 14.8s
362:	test: 0.7725299	best: 0.7726149 (243)	total: 12.3s	remaining: 14.8s
363:	test: 0.7725204	best: 0.7726149 (243)	total: 12.3s	remaining: 14.7s
364:	test: 0.7725191	best: 0.7726149 (243)	total: 12.3s	remaining: 14.7s
365:	test: 0.7725226	best: 0.7726149 (243)	total: 12.3s	remaining: 14.6s
366:	test: 0.7725226	best: 0.7726149 (243)	total: 12.4s	r

467:	test: 0.7725797	best: 0.7729370 (399)	total: 15.3s	remaining: 10.8s
468:	test: 0.7725777	best: 0.7729370 (399)	total: 15.3s	remaining: 10.8s
469:	test: 0.7725784	best: 0.7729370 (399)	total: 15.3s	remaining: 10.8s
470:	test: 0.7725781	best: 0.7729370 (399)	total: 15.4s	remaining: 10.7s
471:	test: 0.7726163	best: 0.7729370 (399)	total: 15.4s	remaining: 10.7s
472:	test: 0.7726163	best: 0.7729370 (399)	total: 15.4s	remaining: 10.7s
473:	test: 0.7726026	best: 0.7729370 (399)	total: 15.4s	remaining: 10.6s
474:	test: 0.7726136	best: 0.7729370 (399)	total: 15.5s	remaining: 10.6s
475:	test: 0.7726133	best: 0.7729370 (399)	total: 15.5s	remaining: 10.6s
476:	test: 0.7726132	best: 0.7729370 (399)	total: 15.5s	remaining: 10.5s
477:	test: 0.7726138	best: 0.7729370 (399)	total: 15.6s	remaining: 10.5s
478:	test: 0.7726082	best: 0.7729370 (399)	total: 15.6s	remaining: 10.4s
479:	test: 0.7726042	best: 0.7729370 (399)	total: 15.6s	remaining: 10.4s
480:	test: 0.7726026	best: 0.7729370 (399)	total: 1

581:	test: 0.7728634	best: 0.7729505 (570)	total: 19.6s	remaining: 7.33s
582:	test: 0.7728526	best: 0.7729505 (570)	total: 19.6s	remaining: 7.29s
583:	test: 0.7729105	best: 0.7729505 (570)	total: 19.6s	remaining: 7.26s
584:	test: 0.7729031	best: 0.7729505 (570)	total: 19.7s	remaining: 7.24s
585:	test: 0.7729031	best: 0.7729505 (570)	total: 19.7s	remaining: 7.2s
586:	test: 0.7729040	best: 0.7729505 (570)	total: 19.8s	remaining: 7.18s
587:	test: 0.7729008	best: 0.7729505 (570)	total: 19.8s	remaining: 7.15s
588:	test: 0.7728860	best: 0.7729505 (570)	total: 19.9s	remaining: 7.12s
589:	test: 0.7728861	best: 0.7729505 (570)	total: 19.9s	remaining: 7.09s
590:	test: 0.7728873	best: 0.7729505 (570)	total: 19.9s	remaining: 7.05s
591:	test: 0.7728865	best: 0.7729505 (570)	total: 20s	remaining: 7.02s
592:	test: 0.7728906	best: 0.7729505 (570)	total: 20s	remaining: 6.99s
593:	test: 0.7728849	best: 0.7729505 (570)	total: 20s	remaining: 6.95s
594:	test: 0.7728874	best: 0.7729505 (570)	total: 20.1s	re

697:	test: 0.7731102	best: 0.7731275 (689)	total: 23s	remaining: 3.36s
698:	test: 0.7731120	best: 0.7731275 (689)	total: 23s	remaining: 3.33s
699:	test: 0.7731144	best: 0.7731275 (689)	total: 23.1s	remaining: 3.29s
700:	test: 0.7731160	best: 0.7731275 (689)	total: 23.1s	remaining: 3.26s
701:	test: 0.7731075	best: 0.7731275 (689)	total: 23.1s	remaining: 3.23s
702:	test: 0.7731079	best: 0.7731275 (689)	total: 23.1s	remaining: 3.19s
703:	test: 0.7730724	best: 0.7731275 (689)	total: 23.2s	remaining: 3.16s
704:	test: 0.7730743	best: 0.7731275 (689)	total: 23.2s	remaining: 3.13s
705:	test: 0.7730736	best: 0.7731275 (689)	total: 23.3s	remaining: 3.1s
706:	test: 0.7730730	best: 0.7731275 (689)	total: 23.3s	remaining: 3.06s
707:	test: 0.7730661	best: 0.7731275 (689)	total: 23.3s	remaining: 3.03s
708:	test: 0.7730712	best: 0.7731275 (689)	total: 23.3s	remaining: 3s
709:	test: 0.7730717	best: 0.7731275 (689)	total: 23.4s	remaining: 2.96s
710:	test: 0.7730678	best: 0.7731275 (689)	total: 23.4s	rem

In [270]:
y_preds = c.predict(X_test)

In [272]:
from sklearn.metrics import classification_report

In [274]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.64      0.75      0.69      7311
           1       0.78      0.67      0.72      9380

   micro avg       0.70      0.70      0.70     16691
   macro avg       0.71      0.71      0.70     16691
weighted avg       0.71      0.70      0.70     16691

